## Assignement: CA4 - DTMF Decoder
Name: Christopher Yonek (11/12/21)  Professor: Dr. Boncelet

Build a DTMF decoder. Your code should accept a tonal sequence and output a string of key presses.


Rules:

1) Use a sampling rate of 8000 samples per second.

2) Use a set of bandpass filters, not a FFT.

3) For full credit, use FIR filters of length 31 or less or second order (two pole) IIR filters. You get partial credit for working implementations that use larger filters. You may assume the silence and tonal periods are both multiples of 400 samples. (This is a simplifying assumption.)

4) You may not assume the tones or the silence periods are the same length throughout the sequence, i.e., the first tone may have a different length than the second one, etc.


In [88]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import argparse
from IPython.display import Audio
from sklearn.cluster import KMeans
from statistics import mode

## DTMF Encoder

In [95]:
Fs = 8000
#define the keypad and its frequencies
rowfreqs = [697, 770, 852, 941]
colfreqs = [1209, 1336, 1477, 1633]
buttons = {'1':(0,0), '2':(0,1), '3':(0,2),# 'A':(0,3),
           '4':(1,0), '5':(1,1), '6':(1,2),# 'B':(1,3),
           '7':(2,0), '8':(2,1), '9':(2,2),# 'C':(2,3), 'D':(3,3), 
           '0':(3,1)}#,'#':(3,2),'*':(3,0)}
def DtmfEncodeDigit(phonenumber, dur=0.5, silencedur=0.1, Fs=8000, dither=False):
    """return the DTMF tones for a phone number"""
    t = np.linspace(0,dur,int(dur*Fs),endpoint=False)
    silence = np.zeros(int(0.5*Fs))
    
    sounds = []
    for key in phonenumber:
        if key.upper() in buttons:
            r,c = buttons[key.upper()]
            fr, fc = rowfreqs[r], colfreqs[c]
            if dither: #change the frequencies
                fr += dither*(rnd.rand()-0.5)
                fc += dither*(rnd.rand()-0.5)
            #print key, fr, fc
            sounds.append(np.sin(2*np.pi*fr*t)+np.sin(2*np.pi*fc*t))
            sounds.append(silence)
    return np.concatenate(sounds[:-1]) 

def DTMF_encoder(phonenumber,dur=0.5,silencedur=0.1): #Takes in a String and Returns a 2D Array with Encoded Frequencies with
    encodedFreqs = np.zeros((len(phonenumber),4000)) #Each number
    SAMPLE_NO = len(DtmfEncodeDigit(phonenumber[0]))

    for numIndex in range(len(phonenumber)):
        for j in range(SAMPLE_NO):
            encodedFreqs[numIndex][j] = DtmfEncodeDigit(phonenumber[numIndex])[j]
    return encodedFreqs

In [90]:
myphone = '3028313211'
encoded = DtmfEncodeDigit(myphone)
Audio(encoded,rate=Fs)

In [16]:
help = '911'
Audio(DTMF_encoder_digit(help, dur=1,silencedur=.1),rate=Fs)

## DTMF Decoder
* must use pip install KMeans on anaconda prompt

In [91]:
iterations = 0
dtmfDials = np.array(
    [[697,1209], # 1
     [697,1336], # 2
     [697,1477], # 3
     [770,1209], # 4  
     [770,1336], # 5
     [770,1477], # 6
     [852,1209], # 7
     [852,1336], # 8 
     [852,1477], # 9
     [941,1209], # *
     [941,1336]]) # 0
   #  [941,1477]]  #
#)
telephone_keypad = ["{}".format(i) for i in range(1,10)] + ["*","0","#"]

def Bandpass_FIR_filter(dtmfDials, cent):
    return(np.sum((dtmfDials - cent )**2,axis=1))

def GetSpectrum(tone,NFFT=1024,fs=8000):
    Pxx, freqs, bins, im = plt.specgram(tone, NFFT, noverlap=0, Fs=fs);
    plt.close()
    i, k = np.where(np.log10(Pxx )*10 > -50)
    binsTone = bins[k]
    frequencyTone = freqs[i]
    pick  = frequencyTone > 500
    freqSpectrum = np.array([binsTone[pick],frequencyTone[pick]]).T
    return freqSpectrum

def Bandpass_IIR_filter(freqVsTimeArray):
    signalStandardDeviation = np.std(freqVsTimeArray,axis=0)
    signalScale = freqVsTimeArray/signalStandardDeviation
    km = KMeans(n_clusters = 12) #Set low cluster to look through various freqs
    km.fit(signalScale)
    clusterCenter = km.cluster_centers_*signalStandardDeviation      #rescale the cluster center
    clusterCenter = np.array([np.round(clusterCenter[:,0],2),np.round(clusterCenter[:,1],0)]).T #reorder according to the time 
    clusterCenter = clusterCenter[np.argsort(clusterCenter[:,0]),:]
    clusterCenter_xy0 = np.array([clusterCenter[1::2,1],clusterCenter[::2,1]]).T
    clusterCenter_xy = []
    for i in range(clusterCenter_xy0.shape[0]):
        clusterCenter_xy.append(np.sort(clusterCenter_xy0[i]))
    return np.array(clusterCenter_xy)

def GetFrequencySample(clusterCenter_xy,teleKeypad = [],dials = []):  #Takes in a frequncy array and matches them
    freq = []                                                  #to a corresponding number on the keypad
    for i in range(clusterCenter_xy.shape[0]):
        cent = clusterCenter_xy[i]
        distanceTocenter = Bandpass_FIR_filter(dtmfDials, cent)
        iterations = np.argmin(distanceTocenter)
        freq.append(teleKeypad[iterations])
    return freq

def DtmfDecodeDigit(stringDigit):  #Takes a range of guessed frequences and gets the most reoccuring number
    decodedDigit = mode(GetFrequencySample(Bandpass_IIR_filter(
        GetSpectrum(stringDigit)),telephone_keypad,dtmfDials))
    return decodedDigit

def DTMF_decoder(signal): #Takes in a 2D array where rows correspond to digits
    decodedSignal = np.zeros(len(signal)) # And cols correspond to frequency range
    for i in range(len(signal)):
        decodedSignal[i] = DtmfDecodeDigit(signal[i])
    decodedString = str("".join([str(int(i)) for i in decodedSignal]))
    return decodedString # Returns the decoded number in the form of a string

def test_dtmfdecoder(dtmfdecoder, dur=0.5, silencedur=0.1, Fs=8000, **kwds):
    works = False
    sigma = 0.0
    while sigma < 3.1:
        number = ''.join(np.random.permutation(list(buttons.keys())[0:9]))
        tones = DTMF_encoder(number, dur=dur, silencedur=silencedur, **kwds)
        
        for i in range(5):
            noisytone = sigma*np.random.randn(len(tones.flatten()))
            distorted = noisytone+tones.flatten()
            distorted = np.array(distorted).reshape(-1, 4000)
            decoded = DTMF_decoder(distorted)
            if decoded == number:
                works = True
                print(sigma,i)
            else:
                return works, sigma
        sigma += 0.07 #0.1 Sigma causes too much distortion
    return works, sigma
    

## Testing the Decoder

`test_dtmfdecoder` iterates over increasing noise until the `dtmfdecoder` routine fails.  It runs each level 5 times.

In [92]:
test_dtmfdecoder(1) #Run test_dtmfdecoder to verify it works

0.0 0
0.0 1
0.0 2
0.0 3
0.0 4
0.07 0
0.07 1
0.07 2
0.07 3
0.07 4


(True, 0.14)

In [93]:
    a = DtmfEncodeDigit('*') #Check if decoding special digits work
    y = DtmfDecodeDigit(a)
    print(y)

*


In [94]:
'''Additionl Testing for DtmfEncode() and DtmfDecode()'''
number = ''.join(np.random.permutation(list(buttons.keys())[0:10])) #Generate Telephone Number
print('Original Number: ' + number)
tones = DTMF_encoder(number,dur=1,silencedur=0) #Create DTMF tone corresponding to phone number
sigma = 0.01
noisytone = sigma*np.random.randn(len(tones.flatten()))
distorted = noisytone+tones.flatten()
start = 0
for k in range(len(number)): #This decodes each DTMF tone one digit at a time
    print(DTMF_decode_digit(distorted[start:start+4000]))
    start += 4000
distorted = np.array(distorted).reshape(-1, 4000) #Turns a 1D Array into a 2D Array
guessedNumber = DTMF_decoder(distorted)
print('Decoded Number: ' + guessedNumber)
print('Does decoded number match?:')
print((guessedNumber==number)) #Checks if DTMF guess matches original number

Original Number: 7930652814
7
9
3
0
6
5
2
8
1
4
Decoded Number: 7930652814
Does decoded number match?:
True
